# Tree Wrangling

A tree, for our purposes, is an abstract way of organising data where some pieces of data are variously ‘contained in’ or ‘belonging to’ other pieces of data. Trees are used for all kinds of hierarchical data, or to accelerate many kinds of algorithm. In this workshop we're going to look at one way to represent trees as flat arrays, so that we can make best use of the APL primitives for performant tree wrangling.

This workshop is an abridged version of [my online tutorial](https://asherbhs.github.io/apl-site/trees/intro.html) about trees.

More formally, trees are made up of _nodes_. Each node may have some number of _child nodes_, and usually one _parent node_. There is exactly one node in a tree which has no parent, this is the tree’s _root node_. Nodes which share a parent are _sibling nodes_, and nodes with no children are _leaf nodes_.

We will generally draw trees with the root node at the top, and all child nodes arranged below, with a line connecting each child to its parent. For instance, in the following tree, the node labelled $a$ is the root node, and is a parent of $b$, $c$ and $d$ - its children.

![](intro-tree.png)

In [1]:
⎕IO←0
]BOX on

Was OFF

A simple way to represent trees, and the way that is usual for most programmers, is with a nested vector. Each node stores its own data and all its children (who in turn contain their own children, and so on). For instance, the above tree could be represented as

In [2]:
(,'a') ((,'b') (,⊂,'e') (,⊂,'f')) ((,'c') ((,'g') (,⊂,'h') (,⊂,'i') (,⊂,'j'))) (,'d')

┌─┬───────────┬───────────────────┬─┐
│a│┌─┬───┬───┐│┌─┬───────────────┐│d│
│ ││b│┌─┐│┌─┐│││c│┌─┬───┬───┬───┐││ │
│ ││ ││e│││f││││ ││g│┌─┐│┌─┐│┌─┐│││ │
│ ││ │└─┘│└─┘│││ ││ ││h│││i│││j││││ │
│ │└─┴───┴───┘││ ││ │└─┘│└─┘│└─┘│││ │
│ │ ││ │└─┴───┴───┴───┘││ │
│ │ │└─┴───────────────┘│ │
└─┴───────────┴───────────────────┴─┘

A drawback of this representation is that we can't take much advantage of APL's optimised array primitives to get stuff done - we generally have to work on each child recursively, which can be quite slow. 

For this reason, we'll invert the relationship between parent and child. Instead of each parent storing all of its children, we can have each child reference its parent. Specifically, by associating each node with an index into an array, each child can store the index of its parent, forming a _parent vector_:

In [3]:
⍝      ┌─┐─────┐ ┌─┐─┐─┐
⍝      ↓ │     │ ↓ │ │ │
parent←0 0 1 1 0 4 5 5 5 0
⍝      ↑ ↑ │ │ ↑ │       │
⍝      │ └─┘─┘ └─┘       │
⍝      └─────────────────┘

In the nested representation, each node had to store an arbitrary number of children. With a parent vector, each node stores exactly one piece of data, so it flattens it out.

For the root node, we'll see later that it's helpful to store a reference to the very same root node (`0`). There's no reason that `0` should be the _only_ node which loops on itself like this. We can have multiple root nodes giving us multiple trees - a _forest_.

Make sure you can see how the parent vector corresponds to the tree diagram above.

This representation is missing the associated lables for each node. To get around this, we store the labels in a separate vector. Since each node is associated with an index, it's unambiguous which label is associated with each node.

In [4]:
labels←'abefcghijd'
↑(⍳≢parent) parent labels

0 1 2 3 4 5 6 7 8 9
0 0 1 1 0 4 5 5 5 0
a b e f c g h i j d

Here are a few utility functions to help us visualise trees. You don't need to understand this code right now - but it might be fun to look back on later.

In [5]:
Depths←{ ⍝ find the depths of each node in a parent vector
    ⍝ ←: a vector of the depths of each node in the input
    p←⍵    ⍝ input parent vector
    depths←(≢p)⍴0
    StepUp←{ ⍝ step up the tree and increment depths
        q←p[⍵]
        depths+←⍵≠q
        q
    }
    _←StepUp⍣≡⍳≢p
    depths
}

_PrettyPrint_←{ ⍝ renders a tree given labels, box drawing characters, and padding
    ⍝ ←: vector of character matrices, each a labelled rendering of a tree in the forest given by the input parent vector
    labels    ←⍺     ⍝ vector of character matrices giving the labels for each node
    connectors←⍺⍺    ⍝ box drawing characters to render the tree, e.g: '─┌┬┐│┴├┼┤│' (normal, and upstruck)
    spaces    ←⍵⍵    ⍝ number of spaces to pad with between sub-trees
    p         ←⍵     ⍝ parent vector
    d←Depths p
    maxDepth←⌈/d
    results←labels         ⍝ result of rendering each sub-tree, seeded with labels
    maxDepth=0: results    ⍝ avoid the each running on the prototype
    DoFamily←{ ⍝ render and record a sub-tree
        ⍝ ⍺: parent node
        ⍝ ⍵: rendered results of children
        widths←(1⊃⍴)¨⍵                                                                           ⍝ widths of each rendered child
        width←spaces-⍨+/spaces+widths                                                            ⍝ eventual width of the rendered tree       wwwwwww
        centres←(+\0,¯1↓spaces+widths)+¯1+⌈2÷⍨widths                                             ⍝ centres of each rendered sub-tree         ∘ss∘ss∘
        result←width⍴' '                                                                         ⍝ header to be decorated                   '       '
        result[(⊢⊢⍤/⍨((⊃⌽centres)>⊢)∧(⊃centres)<⊢)⍳width]←connectors[0]                          ⍝ add horizontal bar                       ' ───── '
        result[   ⊃ centres]←connectors[1]                                                       ⍝ left end of bar                          '┌───── '
        result[   ⊃⌽centres]←connectors[3]                                                       ⍝ right end of bar                         '┌─────┐'
        result[¯1↓1↓centres]←connectors[2]                                                       ⍝ connectors to intermediate children      '┌──┬──┐'
        result[(1=≢centres)⍴⊃centres]←connectors[4]                                              ⍝ if there's only one child, just make it a lone upstrike
        centre←¯1+⌈2÷⍨width                                                                      ⍝ index of the centre of the rendered tree     ∘
        result[centre]←connectors[5 6 7 8 9][connectors[0 1 2 3 4]⍳result[centre]]               ⍝ connector to the parent                  '┌──┼──┐'
        result⍪←(-spaces)↓⍤1⊃,/,∘(spaces⍴' ')⍤1¨⍵↑¨⍨⌈/≢¨⍵                                        ⍝ pad labels, join under header
        parentResult←⍺⊃results                                                                   ⍝ label of the parent
        parentWidth←1⊃⍴parentResult                                                              ⍝ width of label of parent
        parentCentre←¯1+⌈2÷⍨parentWidth                                                          ⍝ centre of label of parent
        result      ←((centre-parentCentre)⌽parentWidth↑⍤1⊢)⍣(width<parentWidth)⊢result          ⍝ pad and recentre text so far if it's less wide
        parentResult←((parentCentre-centre)⌽      width↑⍤1⊢)⍣(width>parentWidth)⊢parentResult    ⍝ pad and recentre parent label if it's less wide
        result⍪⍨←parentResult                                                                    ⍝ add parent label
        results[⍺]←⊂result                                                                       ⍝ record result
        1
    }
    DoLayer←{ ⍝ render and record all nodes whose children have depth ⍵
        ⍝ ⍵: depth to handle nodes at
        i←⍸d=⍵    ⍝ nodes at this depth
        _←p[i]DoFamily⌸results[i]
        1
    }
    _←DoLayer¨⌽1+⍳maxDepth    ⍝ bottom up accumulation
    results/⍨p=⍳≢p            ⍝ return results at roots only
}

PPV←{⍺←'∘' ⋄   ((≢⍵)⍴⍉⍤⍪⍤⍕¨'∘'@(0=≢¨)⍺)('─┌┬┐│┴├┼┤│'_PrettyPrint_ 1)⍵}    ⍝ vertical
PPH←{⍺←'∘' ⋄ ⍉¨((≢⍵)⍴  ⍪⍤⍕¨'∘'@(0=≢¨)⍺)('│┌├└─┤┬┼┴─'_PrettyPrint_ 0)⍵}    ⍝ horizontal

Now we can see that, indeed, our parent vector represents a tree which is the same shape as our diagram above.

In [6]:
PPV parent        ⍝ without labels
labels PPV parent ⍝ with labels

┌───────────┐
│ ∘ │
│ ┌───┴┬───┐│
│ ∘ ∘ ∘│
│┌┴┐ │ │
│∘ ∘ ∘ │
│ ┌─┼─┐ │
│ ∘ ∘ ∘ │
└───────────┘

┌───────────┐
│ a │
│ ┌───┴┬───┐│
│ b c d│
│┌┴┐ │ │
│e f g │
│ ┌─┼─┐ │
│ h i j │
└───────────┘

We can also draw the tree horizontally.

In [7]:
labels PPH parent

┌───────┐
│ ┌b┬e │
│ │ └f │
│a┤ ┌h│
│ ├c─g┼i│
│ │ └j│
│ └d │
└───────┘

Now let's stretch our muscles a bit with some basic challenges. Write an expression to determine

- which nodes are children of the root node (node `0`);
- which nodes are children of nodes $b$ or $c$;
- which nodes are leaves (have no children).

***SPOILERS BELOW***

Don't

scroll

down

just

yet,

try

the

exercises

first.

In [8]:
⍝ there is, of course, more than one way to solve all of these
labels[⍸parent=0]            ⍝ children of 0
labels[⍸parent∊⍸labels∊'bc'] ⍝ children of b or c
labels[⍸~(⍳≢parent)∊parent]  ⍝ leaves

abcd

efg

efhijd

Let's try some edits now. Can you 'break off' node $c$ and all its descendents, so that we have two trees.

***SPOILERS BELOW***

Don't

scroll

down

just

yet,

try

the

exercises

first.

In [9]:
⍝ the solution is to make node c its own parent, making it a root node
i←labels⍳'c'
parent[i]←i
labels PPV parent

┌─────┬─────┐
│ a │ c │
│ ┌┴─┐│ │ │
│ b d│ g │
│┌┴┐ │┌─┼─┐│
│e f │h i j│
└─────┴─────┘

We can see, just by looking at this forest, which nodes are descendants of $a$ and which are descendants of $c$. To figure this out programmatically, we can repeatedly index by the parent vector until we reach a fixed point (until we start looping at the root).

In [10]:
⊢roots←{parent[⍵]}⍣≡parent
labels[roots]
labels[roots] PPV parent

0 0 0 0 4 4 4 4 4 0

aaaaccccca

┌─────┬─────┐
│ a │ c │
│ ┌┴─┐│ │ │
│ a a│ c │
│┌┴┐ │┌─┼─┐│
│a a │c c c│
└─────┴─────┘

Let's switch to a new tree now.

In [11]:
parent←1 7 4 1 7 6 3 7 6 1
(⍳≢parent) PPV parent

┌─────────┐
│ 7 │
│ ┌┴───┐│
│ 1 4│
│┌──┼──┐ ││
│0 3 9 2│
│ │ │
│ 6 │
│ ┌┴┐ │
│ 5 8 │
└─────────┘

See if you can adapt our root-finding method to find a mask of all the nodes which are node $3$ or its descendants.

***SPOILERS BELOW***

Don't

scroll

down

just

yet,

try

the

exercises

first.

In [12]:
mask←1@3⊢3={parent[⍵]}@{⍵≠3}⍣≡parent
mask PPV parent

┌─────────┐
│ 0 │
│ ┌┴───┐│
│ 0 0│
│┌──┼──┐ ││
│0 1 0 0│
│ │ │
│ 1 │
│ ┌┴┐ │
│ 1 1 │
└─────────┘

Now we have a mask of some nodes, let's say we want to delete them. If we just mask out the parent vector, the parent indices will be all wrong. To fix this, we need to reduce the index of each nodes parent by the number of nodes that appeared before that parent in the old vector, before they were deleted. There are two ways to go about this.

In [13]:
⍝ using a scan
parent/⍨←~mask
parent-←(+\mask)[parent]
(⍸~mask) PPV parent

┌─────┐
│ 7 │
│ ┌┴─┐│
│ 1 4│
│┌┴┐ ││
│0 9 2│
└─────┘

In [14]:
⍝ reset the parent vector so we can see the other way
parent←1 7 4 1 7 6 3 7 6 1

⍝ using interval index
parent/⍨←~mask
parent-←1+(⍸mask)⍸parent
(⍸~mask) PPV parent

┌─────┐
│ 7 │
│ ┌┴─┐│
│ 1 4│
│┌┴┐ ││
│0 9 2│
└─────┘

How to choose between these two methods? Profile them on real data for your use-case and see which is faster.

Let's reset our tree one more time.

In [15]:
parent←1 7 4 1 7 6 3 7 6 1

Can you write some code to delete all the leaves of this tree?

***SPOILERS BELOW***

Don't

scroll

down

just

yet,

try

the

exercises

first.

In [16]:
keep←(⍳≢parent)∊parent
parent/⍨←keep
parent-←(+\~keep)[parent] ⍝ you could also use the other idiom
PPV parent

┌───┐
│ ∘ │
│┌┴┐│
│∘ ∘│
││ │
│∘ │
││ │
│∘ │
└───┘

Now that we understand the basics of working structurally with trees, let's see how to aggregate information from across the whole tree. It's time to start taking advantage of 

- depths
- acummulation (following tutorial)
- exercise - find heights?

Our first step will be to find the _depth_ of each node in our tree - defined as the number of ancestors a node has. Our method will be to step up the tree from each node until we reach a root. The number of steps made from each node will be its depth.

In [19]:
⍝ new tree
parent←0 0 1 2 2 1 0 6 7 7 7 0

depths←0×≢parent
StepUp←{ ⍝ step up the tree and increment depths
    q←parent[⍵]
    depths+←⍵≠q
    q
}
_←StepUp⍣≡⍳≢parent
depths PPV parent

┌─────────────┐
│ 0 │
│ ┌───┴─┬───┐│
│ 1 1 1│
│ ┌┴─┐ │ │
│ 2 2 2 │
│┌┴┐ ┌─┼─┐ │
│3 3 3 3 3 │
└─────────────┘

Now we're ready to aggregate some information. By working one 'level' at a time, we can work our way up the tree, accumulating information as we go:

![](accumulation.gif)

By starting at the deepest level and working upwards, we ensure that everything is collected in the right order.

TODO: explain this

In [23]:
result←⊂⍤,¨'abcdefghijkl'
DoLayer←{
    i←⍸depths=⍵
    parent[i]{result[⍺],←⊂result[⍵]}⌸i
}
DoLayer¨⌽1+⍳⌈/depths
⊃result

┌─┬───────────────────┬───────────────────┬───┐
│a│┌─┬───────────┬───┐│┌─┬───────────────┐│┌─┐│
│ ││b│┌─┬───┬───┐│┌─┐│││g│┌─┬───┬───┬───┐│││l││
│ ││ ││c│┌─┐│┌─┐│││f││││ ││h│┌─┐│┌─┐│┌─┐│││└─┘│
│ ││ ││ ││d│││e│││└─┘│││ ││ ││i│││j│││k││││ │
│ ││ ││ │└─┘│└─┘││ │││ ││ │└─┘│└─┘│└─┘│││ │
│ ││ │└─┴───┴───┘│ │││ │└─┴───┴───┴───┘││ │
│ │└─┴───────────┴───┘│└─┴───────────────┘│ │
└─┴───────────────────┴───────────────────┴───┘